In [8]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
import math
import pandas as pd
from collections import Counter

# 在console内直接生成图像
%matplotlib inline 

### 例 5-1 数据

In [51]:
datasets = [['青年', '否', '否', '一般', '否'],
               ['青年', '否', '否', '好', '否'],
               ['青年', '是', '否', '好', '是'],
               ['青年', '是', '是', '一般', '是'],
               ['青年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '好', '否'],
               ['中年', '是', '是', '好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '好', '是'],
               ['老年', '是', '否', '好', '是'],
               ['老年', '是', '否', '非常好', '是'],
               ['老年', '否', '否', '一般', '否']]
labels = ['年龄', '有工作', '有自己的房子', '信贷情况', '类别']
pd.DataFrame(datasets, columns=labels)

,年龄,有工作,有自己的房子,信贷情况,类别
0,青年,否,否,一般,否
1,青年,否,否,好,否
2,青年,是,否,好,是
3,青年,是,是,一般,是
4,青年,否,否,一般,否
5,中年,否,否,一般,否
6,中年,否,否,好,否
7,中年,是,是,好,是
8,中年,否,是,非常好,是
9,中年,否,是,非常好,是


In [16]:
# 计算熵
def cal_entropy(Y):
    class_cnt = Counter(Y)  # 对Y中的不同类别做计数
    entropy = 0
    tot = len(Y)
    for value in class_cnt.values():
        tmp = value / tot
        entropy -= tmp * math.log(tmp,2)  # 累加 -∑ plog(p)
        
    return entropy

# 计算经验条件熵(条件A下Y的经验熵)
def cal_con_entropy(X, Y, A):
    class_cnt = {}
    for i in range(len(X)):  # 将样本中特征A相同的数据所对应的y放到一个列表中
        if class_cnt.get(X[i][A]) != None:
            class_cnt[X[i][A]].append(Y[i])
        else:
            class_cnt[X[i][A]] = [Y[i]]         
    con_entropy = 0
    tot = len(Y)
    for value in class_cnt.values():  # 条件熵 = 样本中特征A取某值的比例 * A取某值的熵
        con_entropy += len(value) / tot * cal_entropy(value)
        
    return con_entropy

# 计算信息增益
def cal_info_gain(X, Y, A):
    entropy = cal_entropy(Y)
    con_entropy = cal_con_entropy(X, Y, A)
    return entropy - con_entropy

# 计算信息增益比
def cal_info_gain_ratio(X, Y, A=0):
    info_gain = cal_info_gain(X, Y, A)
    cnt = Counter(X[:, A])
    tot = len(X)
    A_entropy = 0
    for value in cnt.values():
        tmp = value / tot
        A_entropy -= tmp * math.log(tmp, 2)
    
    return info_gain / A_entropy

In [17]:
x_train = np.array(datasets)
print(cal_entropy(x_train[:,-1]))
print(cal_con_entropy(x_train[:,0:-1], x_train[:,-1], 0))
print(cal_info_gain(x_train[:,0:-1], x_train[:,-1], 0))
print(cal_info_gain_ratio(x_train[:,0:-1], x_train[:,-1], 0))

0.9709505944546686
0.8879430945988998
0.08300749985576883
0.05237190142858302


In [45]:
# 寻找实例中最多的类别
def find_max_class(node):
    cnt = Counter(node.data[:,-1])
    # 实例中最多的类别作为该结点预测值
    return sorted(cnt.items(), key=lambda x:x[1], reverse=True)[0][0]  

### C4.5 决策树生成算法

In [64]:
class DT_Node:
    def __init__(self, data, indices):
        self.data = data  # 可分类实例
        self.childs = []  # 孩子结点
        self.indices = indices  # 可划分特征的序号列表
        self.predict = None  # 叶子结点表示预测结果，非叶子结点表示分类特征
        
    # 寻找最优分类特征
    def find_opt_feature(self):
        max_igt = 0  # 最大信息增益比
        max_indice = -1  # 最优分类特征的下标
        for i in self.indices:
            igt = cal_info_gain_ratio(self.data[:,0:-1], self.data[:,-1], i)
            if igt > max_igt:  
                max_igt = igt  # 更新最大信息增益比
                max_indice = i  # 更新最优分类特征的下标
        return max_indice, max_igt
                
    
class DTree:
    def __init__(self, data, labels, epsilon=0.1):
        self.epsilon = epsilon  # 下限阈值
        self.labels = labels  # 可划分特征集合
        indices = [i for i in range(len(labels)-1)]  # 初始时根节点的可划分特征为全部特征
        self.root = DT_Node(data, indices)
        
    # 决策树生成算法
    def generate_tree(self, node):
        if node.indices == None:  # 若可选取特征集合为空则为叶子结点
            node.predict = find_max_class(node)
            return
        
        if len(Counter(node.data[:,-1])) == 1:  # 如果所有实例属于同一类，则该结点为叶子结点
            node.predict = node.data[0,-1]  # 修改该叶子结点给出预测值
            return
        
        split_indice, split_igt = node.find_opt_feature();  # 获取最优分类特征下标和信息增益
        if split_igt < self.epsilon:  # 最大信息增益小于阈值，则当前结点设为叶子结点
            node.predict = find_max_class(node)
            return
        node.predict = self.labels[split_indice]
        child_indices = node.indices
        child_indices.remove(split_indice)  # 从可划分特征集合中去除当前分类结点的特征序号
        split_class = np.unique(node.data[:, split_indice])  # 获取当前分类特征的取值集合
        for c in split_class:
            # 获得当前分类特征取值为c的实例集合
            child_data = node.data[[node.data[i][split_indice] == c 
                                    for i in range(len(node.data))]]
#             print(child_data.shape)
            child = DT_Node(child_data, child_indices)
            self.generate_tree(child)  # 递归的生成子树
            node.childs.append(child)  # 将孩子结点加入列表尾部
            
    # 前序遍历
    def Preorder_Traverse(self, node):
        print(node.predict)
        for child in node.childs:
                self.Preorder_Traverse(child)  # 递归遍历孩子结点


In [65]:
dt = DTree(np.array(datasets), labels, epsilon=0.1)
dt.generate_tree(dt.root)
dt.Preorder_Traverse(dt.root)

(9, 5)
(6, 5)
(3, 5)
(6, 5)
有自己的房子
有工作
否
是
是


In [165]:
# 计算基尼指数
def cal_Gini(Y):
    if len(Y) == 0:
        return 0
    tot = len(Y)
    class_cnt = Counter(Y)  # 对Y中的不同类别做计数
    entropy = 0
#     print(Y)
#     print(class_cnt.most_common(1)[0][1])
    p = class_cnt.most_common(1)[0][1] / tot

    return 2 * p * (1-p)  # 二分类下的基尼指数计算
    
# 计算在特征A的条件下的基尼指数
def cal_con_Gini(X, Y, A):
    class_cnt = {}
    for i in range(len(X)):  # 将样本中特征A相同的数据所对应的y放到一个列表中
        if class_cnt.get(X[i][A]) != None:
            class_cnt[X[i][A]].append(Y[i])
        else:
            class_cnt[X[i][A]] = [Y[i]]
    min_gini = 100  # 最小基尼指数
    a = None  # 特征A中最小基尼指数对应的特征值
    tot = len(Y)  # 总共的实例个数
    for key,value in class_cnt.items():  # key为特征取值，value为该取值实例的类别集合
        except_y = Y.copy()  # 除了value以外的类别
        for i in value:
            except_y.remove(i)  # 从Y中依次删掉value中的类别        
        tmp1 = (len(value) / tot) * cal_Gini(value)
        tmp2 = (len(except_y) / tot) * cal_Gini(except_y)
        tmp = tmp1 + tmp2  # 条件基尼指数计算
#         print(tmp)
        if tmp < min_gini:  # 选择最小基尼指数
            a = key
            min_gini = tmp
    return a, min_gini
        

In [160]:
x_train = np.array(datasets)
print(cal_con_Gini(x_train[:,0:-1], x_train[:,-1].tolist(), 2))


0.26666666666666666
0.26666666666666666
('否', 0.26666666666666666)


### CART算法

In [163]:
class DT_Node:
    def __init__(self, data, features):
        self.data = data  # 可分类实例
        self.left = None  # 左孩子
        self.right = None # 右孩子
        self.features = features  # 可划分特征数量
        self.predict = None  # 叶子结点表示预测结果，非叶子结点表示分类特征
        
    # 寻找最优分类特征
    def find_opt_feature(self):
        min_gini = 0  # 最小基尼指数
        opt_gini = 1000  # 最优划分点基尼指数，初始为一个很大的值
        a = None  # 最小基尼指数对应特征值
        opt_a = None  # 最优划分点对应的特征值
        opt_i = -1  # 最优划分点的特征序号
        for i in range(self.features):
            a, min_gini = cal_con_Gini(self.data[:,0:-1], self.data[:,-1].tolist(), i)
            if min_gini < opt_gini:   # 更新最优划分点
                opt_gini = min_gini 
                opt_a = a
                opt_i = i
        return opt_a, opt_gini, opt_i
                
    
class DTree:
    def __init__(self, data, labels, gini_epsilon=0.02, cnt_epsilon=3):
        self.cnt_epsilon = cnt_epsilon  # 样本个数下限阈值
        self.gini_epsilon = gini_epsilon  # 基尼指数下限阈值
        self.labels = labels  # 可划分特征集合
        self.root = DT_Node(data, len(labels)-1)
        
    # 决策树生成算法
    def generate_tree(self, node):
        cur_gini =  cal_Gini(node.data[:,-1].tolist())
        # 样本基尼指数小于阈值或样本个数小于预定阈值，则当前结点设为叶子结点
        if cur_gini < self.gini_epsilon or len(node.data) < self.cnt_epsilon:  
            node.predict = find_max_class(node)
            return
        
        # 获取最优分类的特征点进行“是/否”的二分类
        opt_a, opt_gini, opt_i = node.find_opt_feature();  
        print(opt_gini)

        node.predict = self.labels[opt_i] + ':' + opt_a
        left_data = node.data[[node.data[i][opt_i] == opt_a  # 左孩子为分类特征点取是的样本集合
                                    for i in range(len(node.data))]]
        if len(left_data) != 0:
            node.left = DT_Node(left_data, node.features)
            self.generate_tree(node.left)  # 递归的生成左子树

        right_data = node.data[[node.data[i][opt_i] != opt_a  # 右孩子为分类特征点取否的样本集合
                                for i in range(len(node.data))]]
        if len(right_data) != 0:
            node.right = DT_Node(right_data, node.features)
            self.generate_tree(node.right)  # 递归的生成右子树

    # 前序遍历
    def Preorder_Traverse(self, node):
        if node == None:
            return
        print(node.predict)
        self.Preorder_Traverse(node.left)  # 递归遍历左孩子结点
        self.Preorder_Traverse(node.right)  # 递归遍历右孩子结点
        


In [166]:
dt = DTree(np.array(datasets), labels, gini_epsilon=0.02, cnt_epsilon = 3)
dt.generate_tree(dt.root)
dt.Preorder_Traverse(dt.root)

0.26666666666666666
0.0
有自己的房子:否
有工作:否
否
是
是
